In [ ]:
!pip install --quiet gradio

from typing import List, Dict
import gradio as gr

# =========================
# 1. SBC limits (based on your PRD)
# =========================

SBC_LIMITS = {
    "max_riser_mm": 175,
    "min_riser_mm": 150,
    "min_tread_mm": 275,
    "min_flight_width_mm": 1100,
    "min_landing_width_mm": 1100,
    "min_headroom_landing_mm": 2300,
    "min_headroom_between_flights_mm": 2100,
    "handrail_min_mm": 850,
    "handrail_max_mm": 950,
    "min_guardrail_height_mm": 1100,
    "max_guard_opening_mm": 150
}

def check_rule(name: str, condition: bool, detail: str) -> Dict:
    return {
        "rule": name,
        "pass": bool(condition),
        "details": detail
    }

# =========================
# 2. Rule-based logic (your PRD)
# =========================

def check_stair_sbc(stair: Dict) -> List[Dict]:
    L = SBC_LIMITS
    results = []

    results.append(check_rule(
        f"Riser height between {L['min_riser_mm']}–{L['max_riser_mm']} mm",
        L["min_riser_mm"] <= stair["riser_mm"] <= L["max_riser_mm"],
        f"Input: {stair['riser_mm']} mm"
    ))

    results.append(check_rule(
        f"Tread depth ≥ {L['min_tread_mm']} mm",
        stair["tread_mm"] >= L["min_tread_mm"],
        f"Input: {stair['tread_mm']} mm"
    ))

    results.append(check_rule(
        f"Flight width ≥ {L['min_flight_width_mm']} mm",
        stair["flight_width_mm"] >= L["min_flight_width_mm"],
        f"Input: {stair['flight_width_mm']} mm"
    ))

    results.append(check_rule(
        f"Landing width ≥ {L['min_landing_width_mm']} mm",
        stair["landing_width_mm"] >= L["min_landing_width_mm"],
        f"Input: {stair['landing_width_mm']} mm"
    ))

    results.append(check_rule(
        f"Headroom above landings ≥ {L['min_headroom_landing_mm']} mm",
        stair["headroom_landing_mm"] >= L["min_headroom_landing_mm"],
        f"Input: {stair['headroom_landing_mm']} mm"
    ))

    results.append(check_rule(
        f"Headroom between flights ≥ {L['min_headroom_between_flights_mm']} mm",
        stair["headroom_between_flights_mm"] >= L["min_headroom_between_flights_mm"],
        f"Input: {stair['headroom_between_flights_mm']} mm"
    ))

    results.append(check_rule(
        f"Handrail height {L['handrail_min_mm']}–{L['handrail_max_mm']} mm",
        L["handrail_min_mm"] <= stair["handrail_height_mm"] <= L["handrail_max_mm"],
        f"Input: {stair['handrail_height_mm']} mm"
    ))

    results.append(check_rule(
        f"Guardrail height ≥ {L['min_guardrail_height_mm']} mm",
        stair["guardrail_height_mm"] >= L["min_guardrail_height_mm"],
        f"Input: {stair['guardrail_height_mm']} mm"
    ))

    results.append(check_rule(
        f"Guard opening ≤ {L['max_guard_opening_mm']} mm (150mm sphere not passing)",
        stair["max_guard_opening_mm"] <= L["max_guard_opening_mm"],
        f"Input: {stair['max_guard_opening_mm']} mm"
    ))

    results.append(check_rule(
        "Door projection ≤ 1/2 landing width",
        stair["door_leaf_projection_mm"] <= stair["landing_width_mm"] / 2,
        f"Door: {stair['door_leaf_projection_mm']} mm, Landing: {stair['landing_width_mm']} mm"
    ))

    return results


# =========================
# 3. Output summary (Markdown)
# =========================

def summarize_results_markdown(results: List[Dict]) -> str:
    overall_pass = all(r["pass"] for r in results)
    text = f"## Overall: **{'COMPLIANT ✅' if overall_pass else 'NON-COMPLIANT ❌'}**\n\n"

    text += "### SBC Rule Checks\n"
    for r in results:
        status = "🟢 PASS" if r["pass"] else "🔴 FAIL"
        text += f"- **{status}** — {r['rule']} (_{r['details']}_) \n"

    return text


# =========================
# 4. Gradio UI (no API, no key)
# =========================

def run_checker(riser, tread, flight_w, landing_w,
                head_l, head_f, handrail, guard, opening, door_proj):

    stair = {
        "riser_mm": riser,
        "tread_mm": tread,
        "flight_width_mm": flight_w,
        "landing_width_mm": landing_w,
        "headroom_landing_mm": head_l,
        "headroom_between_flights_mm": head_f,
        "handrail_height_mm": handrail,
        "guardrail_height_mm": guard,
        "max_guard_opening_mm": opening,
        "door_leaf_projection_mm": door_proj,
    }

    results = check_stair_sbc(stair)
    return summarize_results_markdown(results)

with gr.Blocks() as demo:
    gr.Markdown("# 🏛️ SBC Staircase Compliance Checker")
    gr.Markdown("Rule-based tool generated from PRD (no API key required).")

    with gr.Row():
        riser = gr.Number(label="Riser (mm)", value=170)
        tread = gr.Number(label="Tread (mm)", value=280)
        flight_w = gr.Number(label="Flight Width (mm)", value=1200)

    with gr.Row():
        landing_w = gr.Number(label="Landing Width (mm)", value=1200)
        head_l = gr.Number(label="Headroom above Landings (mm)", value=2350)
        head_f = gr.Number(label="Headroom between Flights (mm)", value=2150)

    with gr.Row():
        handrail = gr.Number(label="Handrail Height (mm)", value=900)
        guard = gr.Number(label="Guardrail Height (mm)", value=1100)
        opening = gr.Number(label="Max Guard Opening (mm)", value=140)

    door_proj = gr.Number(label="Door Leaf Projection (mm)", value=400)

    btn = gr.Button("Check SBC Compliance")

    output = gr.Markdown()

    btn.click(
        run_checker,
        inputs=[riser, tread, flight_w, landing_w, head_l, head_f, handrail, guard, opening, door_proj],
        outputs=output
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e65804514dbdd59db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --quiet gradio

import gradio as gr
from typing import Dict, List

SBC_LIMITS = {
    "max_riser_mm": 175,
    "min_riser_mm": 150,
    "min_tread_mm": 275,
    "min_flight_width_mm": 1100,
    "min_landing_width_mm": 1100,
    "min_headroom_landing_mm": 2300,
    "min_headroom_between_flights_mm": 2100,
    "handrail_min_mm": 850,
    "handrail_max_mm": 950,
    "min_guardrail_height_mm": 1100,
    "max_guard_opening_mm": 150
}

def check_rule(name: str, condition: bool, detail: str) -> Dict:
    return {
        "rule": name,
        "pass": condition,
        "details": detail
    }

def check_stair_sbc(stair: Dict) -> List[Dict]:
    L = SBC_LIMITS
    results = []

    results.append(check_rule(
        f"Riser height between {L['min_riser_mm']}–{L['max_riser_mm']} mm",
        L["min_riser_mm"] <= stair["riser_mm"] <= L["max_riser_mm"],
        f"{stair['riser_mm']} mm"
    ))

    results.append(check_rule(
        f"Tread depth ≥ {L['min_tread_mm']} mm",
        stair["tread_mm"] >= L["min_tread_mm"],
        f"{stair['tread_mm']} mm"
    ))

    results.append(check_rule(
        f"Flight width ≥ {L['min_flight_width_mm']} mm",
        stair["flight_width_mm"] >= L["min_flight_width_mm"],
        f"{stair['flight_width_mm']} mm"
    ))

    results.append(check_rule(
        f"Landing width ≥ {L['min_landing_width_mm']} mm",
        stair["landing_width_mm"] >= L["min_landing_width_mm"],
        f"{stair['landing_width_mm']} mm"
    ))

    results.append(check_rule(
        f"Headroom above landings ≥ {L['min_headroom_landing_mm']} mm",
        stair["headroom_landing_mm"] >= L["min_headroom_landing_mm"],
        f"{stair['headroom_landing_mm']} mm"
    ))

    results.append(check_rule(
        f"Headroom between flights ≥ {L['min_headroom_between_flights_mm']} mm",
        stair["headroom_between_flights_mm"] >= L["min_headroom_between_flights_mm"],
        f"{stair['headroom_between_flights_mm']} mm"
    ))

    results.append(check_rule(
        f"Handrail height {L['handrail_min_mm']}–{L['handrail_max_mm']} mm",
        L["handrail_min_mm"] <= stair["handrail_height_mm"] <= L["handrail_max_mm"],
        f"{stair['handrail_height_mm']} mm"
    ))

    results.append(check_rule(
        f"Guardrail height ≥ {L['min_guardrail_height_mm']} mm",
        stair["guardrail_height_mm"] >= L["min_guardrail_height_mm"],
        f"{stair['guardrail_height_mm']} mm"
    ))

    results.append(check_rule(
        f"Guard opening ≤ {L['max_guard_opening_mm']} mm",
        stair["max_guard_opening_mm"] <= L["max_guard_opening_mm"],
        f"{stair['max_guard_opening_mm']} mm"
    ))

    results.append(check_rule(
        "Door projection ≤ 1/2 landing width",
        stair["door_leaf_projection_mm"] <= stair["landing_width_mm"] / 2,
        f"{stair['door_leaf_projection_mm']} mm"
    ))

    return results

def summarize_results_markdown(results: List[Dict]) -> str:
    overall_pass = all(r["pass"] for r in results)
    text = f"## Overall: **{'COMPLIANT ✅' if overall_pass else 'NON-COMPLIANT ❌'}**\n\n"
    text += "### SBC Rule Checks\n"
    for r in results:
        status = "🟢 PASS" if r["pass"] else "🔴 FAIL"
        text += f"- **{status}** — {r['rule']} (_{r['details']}_) \n"
    return text

def run_checker(riser, tread, flight_w, landing_w,
                head_l, head_f, handrail, guard, opening, door_proj):

    stair = {
        "riser_mm": riser,
        "tread_mm": tread,
        "flight_width_mm": flight_w,
        "landing_width_mm": landing_w,
        "headroom_landing_mm": head_l,
        "headroom_between_flights_mm": head_f,
        "handrail_height_mm": handrail,
        "guardrail_height_mm": guard,
        "max_guard_opening_mm": opening,
        "door_leaf_projection_mm": door_proj,
    }

    results = check_stair_sbc(stair)
    return summarize_results_markdown(results)

with gr.Blocks() as demo:
    gr.Markdown("# 🏛️ SBC Staircase Compliance Checker")

    with gr.Row():
        riser = gr.Number(label="Riser (mm)", value=170)
        tread = gr.Number(label="Tread (mm)", value=280)
        flight_w = gr.Number(label="Flight Width (mm)", value=1200)

    with gr.Row():
        landing_w = gr.Number(label="Landing Width (mm)", value=1200)
        head_l = gr.Number(label="Headroom above Landings (mm)", value=2350)
        head_f = gr.Number(label="Headroom between Flights (mm)", value=2150)

    with gr.Row():
        handrail = gr.Number(label="Handrail Height (mm)", value=900)
        guard = gr.Number(label="Guardrail Height (mm)", value=1100)
        opening = gr.Number(label="Max Guard Opening (mm)", value=140)

    door_proj = gr.Number(label="Door Leaf Projection (mm)", value=400)

    btn = gr.Button("Check SBC Compliance")
    output = gr.Markdown()

    btn.click(
        run_checker,
        inputs=[riser, tread, flight_w, landing_w, head_l, head_f, handrail, guard, opening, door_proj],
        outputs=output
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ef51c09c26dfd3cf5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
